### Mortgage Calculator 

In [9]:
M = 10**6
K = 10**3
months = 12

def dollar(s):
    return '${:,.2f}'.format(s)

def rmb(s):
    return '￥{:,.2f}'.format(s * 7)

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [10]:
# part 2: show basic report
import pandas as pd
from ipywidgets import interact
from IPython.display import display, HTML

def basic_report(
    price, 
    down, 
    closing_cost=42830,
    num_units=3,
    unit_income=2500,
    mortgage=7277,
    repair=50*K,
    insurance=480,
    property_tax=345):
    """
    use https://smartasset.com/mortgage/closing-costs to compute closing-costs
    use zillow or online calculator for monthly mortgage payment
    """
    try:
        closing_cost, mortgage, repair, unit_income, insurance, property_tax, purchase_price, down_payment_rate = \
            int(closing_cost), int(mortgage), int(repair), int(unit_income), int(insurance), \
            int(property_tax), int(float(price)), down / 100

        vacancy_rate = 5 / 100
        repair_rate = 5 / 100
        property_management_rate = 5 / 100

        down_payments, loan = purchase_price * down_payment_rate, purchase_price * (1 - down_payment_rate)
        pre_rent_holding_cost = mortgage * 2

        total_cost = purchase_price+closing_cost+pre_rent_holding_cost+repair
        total_cash_needed = total_cost - loan # or down-payment, closing-cost, pre-rent-holding, repair
        cash_table = pd.DataFrame({
            "expense type": ["Down Payment", "Closing Cost", "Pre-Rent Holding", "Repair", "(Total)"],
            "amount" : [dollar(down_payments), dollar(closing_cost), dollar(pre_rent_holding_cost), 
                            dollar(repair), dollar(total_cash_needed)]
        })
        display(HTML(cash_table.to_html()))
        total_monthly_income = num_units * unit_income

        vacancy = total_monthly_income * vacancy_rate
        repair = total_monthly_income * repair_rate
        property_management = total_monthly_income * property_management_rate

        estimated_monthly_expense = mortgage + \
        property_tax + insurance + vacancy + repair + property_management
        expense_table = pd.DataFrame({
            "expense type": ["Mortgage", "Repair", "Insurance", "Property tax", "Vacancy", "Property Managment", "(Total)"],
            "amount" : [dollar(mortgage), dollar(repair), dollar(insurance), 

                        dollar(property_tax), dollar(vacancy), dollar(property_management),
                            dollar(estimated_monthly_expense)]
        })
        display(HTML(expense_table.to_html()))
        monthly_cash_flow = total_monthly_income - estimated_monthly_expense
        ROI = monthly_cash_flow * months / total_cash_needed
        
        return pd.DataFrame(
            {
                "basic info": [
                    "cash needed in dollar",
                    "cash need in rmb",
                    "cash flow (net income)",
                    "gross income",
                    "estimated expense (month)",
                    "percentage covered"
                ],
                "info" : [
                    dollar(total_cash_needed),
                    rmb(total_cash_needed),
                    dollar(monthly_cash_flow),
                    dollar(total_monthly_income),
                    dollar(estimated_monthly_expense),
                    str(round(total_monthly_income / estimated_monthly_expense, 2)*100) + "%"
                           ]}
        )
    except:
        print("Please pass in valid values!")

test = interact(
    basic_report, 
    price="18000000", 
    down=(5, 50, 5),
    closing_cost = "40000",
    num_units = (1, 12, 1),
    mortgage = "7000",
    repair = "50000",
    unit_income = "2500",
    insurance = "500",
    property_tax = "400")

interactive(children=(Text(value='18000000', description='price'), IntSlider(value=25, description='down', max…

<!-- ### Monthly Mortgage Payment Formula

$$M = P[\frac{r(1+r)^n}{(1+r)^n)-1}]$$

+ `M` = the total monthly mortgage payment.
+ `P` = the principal loan amount.
+ `r` = your monthly interest rate. Lenders provide you an annual rate so you’ll need to divide that figure by $12$ (the number of months in a year) to get the monthly rate. If your interest rate is $5\%$, your monthly rate would be $0.004167$ ($\frac{5\%}{12}=0.004167$)
+ `n` = number of payments over the loan’s lifetime. Multiply the number of years in your loan term by $12$ (the number of months in a year) to get the number of payments for your loan. For example, a $30$-year fixed mortgage would have $360$ payments ($30*12=360$)

[credit link](https://www.bankrate.com/calculators/mortgages/mortgage-calculator.aspx) -->

<!-- TO DO LIST

+ formula of closing cost
+ formula of morgage
+ formula for repair
+ formula for insurance
+ formula property tax -->